In [1]:
import os
import sys
sys.path.append('..')

from utils.dataiter import Dataset
from utils.preprocessing import *
from models.baseline.random_model import random_prediction_model
from models.model.FFNN import FFNN
from core.config import raw_features
from utils.evaluate import calculate_ctr, compute_rce, average_precision_score

import core.config as conf
from tqdm import tqdm

## predict

In [2]:
def evaluate_test_set():
    path = '/hdd/twitter/raw_lzo/' # ./test
    part_files = sorted([os.path.join(path, f) for f in os.listdir(path) if 'part' in f])[:3]
    model_path = '/hdd/models/ffnn_pkl/'
    ds = Dataset()
    with open('../submission/results.csv', 'w') as output:
        for file in tqdm(part_files):
            test_df = read_data(file)
            test_df_reply = ds.pickle_matching(ds.preprocess(test_df, TARGET_id=conf.REPLY), TARGET_id=conf.REPLY)
            test_df_retweet = ds.pickle_matching(ds.preprocess(test_df, TARGET_id=conf.RETWEET), TARGET_id=conf.RETWEET)
            test_df_comment = ds.pickle_matching(ds.preprocess(test_df, TARGET_id=conf.COMMNET), TARGET_id=conf.COMMNET)
            test_df_like = ds.pickle_matching(ds.preprocess(test_df, TARGET_id=conf.LIKE), TARGET_id=conf.LIKE)

            pred_reply = FFNN(test_df_reply, conf.REPLY).predict(model_path) 
            pred_retweet = FFNN(test_df_retweet, conf.RETWEET).predict(model_path) 
            pred_comment = FFNN(test_df_comment, conf.COMMNET).predict(model_path) 
            pred_like = FFNN(test_df_like, conf.LIKE).predict(model_path) 

            with open(file, 'r') as f:
                for i, line in enumerate(f.readlines()):
                    tweet_id, user_id, features = parse_input_line(line)
                    reply_pred = pred_reply[i][0]
                    retweet_pred = pred_retweet[i][0]
                    quote_pred = pred_comment[i][0]
                    fav_pred = pred_like[i][0]
                    output.write(f'{tweet_id},{user_id},{reply_pred},{retweet_pred},{quote_pred},{fav_pred}\n')

            del test_df

In [5]:
evaluate_test_set()

  0%|          | 0/3 [00:00<?, ?it/s]['creator_following_count', 'creator_follower_count', 'engager_follower_count', 'engager_following_count', 'dt_dow', 'dt_hour', 'len_domains', 'creator_main_language', 'engager_main_language', 'engager_feature_number_of_previous_reply_engagement', 'number_of_engagements_positive']
         creator_following_count  creator_follower_count  \
0                            662                    2473   
1                            228                 4418640   
2                           3685                  219715   
3                          13511                 2388283   
4                            720                     414   
...                          ...                     ...   
3033897                     2332                    2230   
3033898                      687                   90842   
3033899                      328                     608   
3033900                      888                 6777417   
3033901              

ValueError: in user code:

    /home/hyez/anaconda3/envs/dask-cudf/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1478 predict_function  *
        return step_function(self, iterator)
    /home/hyez/anaconda3/envs/dask-cudf/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/hyez/anaconda3/envs/dask-cudf/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/hyez/anaconda3/envs/dask-cudf/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/hyez/anaconda3/envs/dask-cudf/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)

## evaluate

In [8]:
path = '/hdd/twitter/raw_lzo/part-00000'
df = read_data(path)

In [9]:
df['reply_timestamp']   = df['reply_timestamp'].fillna(0)
df['retweet_timestamp'] = df['retweet_timestamp'].fillna(0)
df['comment_timestamp'] = df['comment_timestamp'].fillna(0)
df['like_timestamp']    = df['like_timestamp'].fillna(0)

df['reply_timestamp']   = df['reply_timestamp'].astype(np.uint32)
df['retweet_timestamp'] = df['retweet_timestamp'].astype(np.uint32)
df['comment_timestamp'] = df['comment_timestamp'].astype(np.uint32)
df['like_timestamp']    = df['like_timestamp'].astype(np.uint32)

df['reply'] = df['reply_timestamp'].apply(lambda x: 1 if x > 0 else 0).astype(np.int32)
df['retweet'] = df['retweet_timestamp'].apply(lambda x: 1 if x > 0 else 0).astype(np.int32)
df['comment'] = df['comment_timestamp'].apply(lambda x: 1 if x > 0 else 0).astype(np.int32)
df['like'] = df['like_timestamp'].apply(lambda x: 1 if x > 0 else 0).astype(np.int32) 

In [10]:
labels = df[conf.target]


In [15]:
pred = pd.read_csv('../submission/results.csv', names=['tweet_id','user_id','reply_pred','retweet_pred','comment_pred','like_pred'])

In [17]:
reply_rce = compute_rce(pred['reply_pred'], labels['reply'])
rewtwrce = compute_rce(pred['retweet_pred'], labels['retweet'])
comment_rce = compute_rce(pred['comment_pred'], labels['comment'])
retweet_rce = compute_rce(pred['retweet_pred'], labels['retweet'])
print(reply_rce, rewtwrce, comment_rce, retweet_rce)

NameError: name 'compute_rce' is not defined

In [16]:
pred

,tweet_id,user_id,reply_pred,retweet_pred,comment_pred,like_pred


In [ ]:
reply_ap = average_precision_score(labels['reply'], pred['reply_pred'])
retweet_ap = average_precision_score(labels['retweet'], pred['retweet_pred'])
comment_ap = average_precision_score(labels['comment'], pred['comment_pred'])
retweet_ap = average_precision_score(labels['retweet'], pred['retweet_pred'])
print(reply_ap, retweet_ap, comment_ap, retweet_ap)